In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, VBox, HBox, interactive_output
from IPython.display import display

# Radii settings
r_safe_social = 1.0
r_safe_eco = 2.0
r_max = 3.0

# Indicator positions
_social_angles = [0, np.pi/2, np.pi, 3*np.pi/2]
_eco_angles = [np.pi/4, 3*np.pi/4, 5*np.pi/4, 7*np.pi/4]

# Labels
titles_social = ["Income", "Worker Safety", "Gender Equality", "Education"]
titles_eco = ["Climate Change", "Freshwater Usage", "Chemical Pollution", "Microplastic Pollution"]

def plot_combined_green_ring(
    income_safe, income_current,
    workersafe_safe, workersafe_current,
    gender_safe, gender_current,
    education_safe, education_current,
    climate_safe, climate_current,
    freshwater_safe, freshwater_current,
    chemical_safe, chemical_current,
    microplastic_safe, microplastic_current
):
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})

    # Green safe zone area
    theta = np.linspace(0, 2*np.pi, 400)
    ax.fill_between(theta, r_safe_social, r_safe_eco, color='green', alpha=0.25)

    # Draw thin green boundary circles
    ax.plot(theta, np.full_like(theta, r_safe_social), color='green', linewidth=1.5, linestyle='--')
    ax.plot(theta, np.full_like(theta, r_safe_eco), color='green', linewidth=1.5, linestyle='--')

    # Labels
    ax.text(np.pi/2, 1.5, 'Safe Zone', ha='center', va='center', fontsize=14, fontweight='bold')
    ax.text(np.pi, 0.55, 'Social Shortfall', ha='center', va='center', color='red', fontsize=9)
    ax.text(np.pi, 2.55, 'Ecological Overshoot', ha='center', va='center', color='red', fontsize=12)

    # Bar width
    bw = 0.3

    # Social indicators
    safe_social = [income_safe, workersafe_safe, gender_safe, education_safe]
    curr_social = [income_current, workersafe_current, gender_current, education_current]
    for i, angle in enumerate(_social_angles):
        s, c = safe_social[i], curr_social[i]
        if c >= s:
            ax.plot(angle, r_safe_social, 'o', color='green', markersize=8)
        else:
            r_c = (c/s) * r_safe_social if s else 0
            ax.bar(angle, r_safe_social - r_c, width=bw, bottom=r_c,
                   color='red', alpha=0.6)
        ax.text(angle, r_safe_social - 0.15, f"{titles_social[i]}\n{c}/{s}", ha='center', va='center', fontsize=7)

    # Ecological indicators
    safe_eco = [climate_safe, freshwater_safe, chemical_safe, microplastic_safe]
    curr_eco = [climate_current, freshwater_current, chemical_current, microplastic_current]
    for i, angle in enumerate(_eco_angles):
        s, c = safe_eco[i], curr_eco[i]
        if i == 0:  # Climate Change (safe if >1%)
            if c >= s:
                ax.plot(angle, r_safe_eco, 'o', color='green', markersize=8)
            else:
                r_c = (c/s) * r_safe_eco if s else 0
                ax.bar(angle, r_safe_eco - r_c, width=bw, bottom=r_c,
                       color='red', alpha=0.6)
        elif i == 3:  # Microplastic Pollution (safe if <30%)
            if c <= s:
                ax.plot(angle, r_safe_eco, 'o', color='green', markersize=8)
            else:
                overshoot = (c - s) / (100 - s)
                r_c = r_safe_eco + overshoot * (r_max - r_safe_eco)
                ax.bar(angle, r_c - r_safe_eco, width=bw, bottom=r_safe_eco,
                       color='red', alpha=0.5 + 0.5 * overshoot)
        else:  # Freshwater & Chemical Pollution (safe if >50%)
            if c >= s:
                ax.plot(angle, r_safe_eco, 'o', color='green', markersize=8)
            else:
                overshoot = (s - c) / s
                r_c = r_safe_eco + overshoot * (r_max - r_safe_eco)
                ax.bar(angle, r_c - r_safe_eco, width=bw, bottom=r_safe_eco,
                       color='red', alpha=0.5 + 0.5 * overshoot)
        ax.text(angle, r_safe_eco + 0.2, f"{titles_eco[i]}\n{c}/{s}", ha='center', va='center', fontsize=7)

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylim(0, r_max)
    plt.show()

# Slider controls
controls = {}
rows = []
params = [
    ('income', 50), ('workersafe', 50), ('gender', 40), ('education', 50),
    ('climate', 1), ('freshwater', 50), ('chemical', 50), ('microplastic', 30)
]
for name, default in params:
    safe_slider = FloatSlider(value=default, min=0, max=100, step=1, description=f'{name.capitalize()} Safe')
    curr_slider = FloatSlider(value=default, min=0, max=100, step=1, description=f'{name.capitalize()} Curr')
    controls[f'{name}_safe'] = safe_slider
    controls[f'{name}_current'] = curr_slider
    rows.append(HBox([safe_slider, curr_slider]))

# Show plot
interactive_plot = interactive_output(plot_combined_green_ring, controls)
display(VBox(rows), interactive_plot)


Output()